In [1]:
import pandas as pd
import numpy as np
data=pd.read_excel("Copy of 500_MT_BERT_Preds _ Accuracy Analysis.xlsx",sheet_name="BERT_ENG_labels")

In [2]:
# data.head()

In [3]:
data=data[['id','summaries','exact_matched_patt_contextual','BERT_Tags','Eng_Label','TP_entries_eng','FP_entries_eng','FN_entries_eng','FP corrected']]

data=data.loc[:499]

In [4]:
data.head()

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label,TP_entries_eng,FP_entries_eng,FN_entries_eng,FP corrected
0,-1zMVjZsVTSaqEUPGqaTqA_0000,i have a masters degree in computational finan...,['java'],['java'],['java'],['java'],[],[],NaN
1,0BEpuKXd00LyMNQDgcqUZw_0000,i am a strategic hr leader with single and mul...,['courion'],[],[],[],[],[],NaN
2,0f9VnfZFwZiyd30xGBK9Ig_0000,multitalented creative marketing and design pr...,['adobe'],"['adobe', 'beacon']",['adobe'],['adobe'],['beacon'],[],NaN
3,0GTKKIK9kj-FVYLEkrqqcA_0000,it professional with an overall 16 years of te...,['wipro'],[],[],[],[],[],NaN
4,0iMM991ETr11zJf6ABimew_0000,an innovative resultdriven leader with a prove...,['paypal'],[],[],[],[],[],NaN


In [5]:
# for i, l in enumerate(data['FP_entries_eng'] ):
#     print("list",i,"is",type(l))

In [6]:
data['FP corrected'] = data['FP corrected'].fillna('[]')
data['FP_entries_eng'] = data['FP_entries_eng'].fillna('[]')
data['FP corrected'] = data['FP corrected'].apply(eval) 
data['TP_entries_eng']= data['TP_entries_eng'].apply(eval)
data['FP_entries_eng']= data['FP_entries_eng'].apply(eval)
data['FN_entries_eng']= data['FN_entries_eng'].apply(eval)
data['exact_matched_patt_contextual'] =data['exact_matched_patt_contextual'] .apply(eval)

In [7]:
data['TP_entries_eng'].str.len().sum()

143

In [8]:
data['FP_entries_eng'].str.len().sum()

30

In [9]:
#corrected fp will move in tp entry
def combine(lis1, lis2):
    return list(set(lis1 + lis2))

data['exact_matched_patt_contextual'] = data.apply(
    lambda x: combine(x['exact_matched_patt_contextual'], x["FP corrected"]), 
    axis = 1)
data['TP_entries_eng'] = data.apply(
    lambda x: combine(x['TP_entries_eng'], x["FP corrected"]), 
    axis = 1)

In [10]:
data['TP']=data['TP_entries_eng'].str.len()
data['FP']=data['FP_entries_eng'].str.len()
data['FN']=data['FN_entries_eng'].str.len()

In [11]:
data['FP corrected'].str.len().sum()

3

In [12]:
data['TP_entries_eng'].str.len().sum()

146

In [15]:
data['FP_entries_eng'].str.len().sum()

30

In [16]:
data.drop(columns=['FP corrected'])

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label,TP_entries_eng,FP_entries_eng,FN_entries_eng,TP,FP,FN
0,-1zMVjZsVTSaqEUPGqaTqA_0000,i have a masters degree in computational finan...,[java],['java'],['java'],[java],[],[],1,0,0
1,0BEpuKXd00LyMNQDgcqUZw_0000,i am a strategic hr leader with single and mul...,[courion],[],[],[],[],[],0,0,0
2,0f9VnfZFwZiyd30xGBK9Ig_0000,multitalented creative marketing and design pr...,[adobe],"['adobe', 'beacon']",['adobe'],[adobe],[beacon],[],1,1,0
3,0GTKKIK9kj-FVYLEkrqqcA_0000,it professional with an overall 16 years of te...,[wipro],[],[],[],[],[],0,0,0
4,0iMM991ETr11zJf6ABimew_0000,an innovative resultdriven leader with a prove...,[paypal],[],[],[],[],[],0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
495,zun1u5pEP6XGEgB8NnH5Cw_0000,after teaching at hogeschool rotterdam i found...,[adobe],[],['adobe'],[],[],[adobe],0,0,1
496,zYOI4JmmTDiT023cU9g7XA_0000,having spent 8 months in 2011 working with dan...,[rsm],[],[],[],[],[],0,0,0
497,ZzIyes8qbSx2AXxe4cSPlA_0000,after 15 years of successful sales and recruit...,[casa],[],[],[],[],[],0,0,0
498,NaN,experienced information technology professiona...,[teradata],[],[],[],[],[],0,0,0


In [17]:
def get_precision(tp,fp):
    
    if tp+fp ==0:
        precision = 0
    else:
        precision = tp/(tp+fp)
    
    return precision

def get_recall(tp,fn):
    
    if tp+fn ==0:
        recall = 0
    else:
        recall = tp/(tp+fn)
    
    return recall

def get_f_score(precision, recall):
    if precision+recall == 0:
        f_score = 0
    else:
        f_score = 2*(precision*recall)/(precision+recall)
        
    return f_score

data['Precision'] = data.apply(lambda x: get_precision(x["TP"], x["FP"]), axis = 1)

data["Recall"] = data.apply(lambda x: get_recall(x["TP"], x["FN"]), axis = 1)

data["f_score"] = data.apply(lambda x: get_f_score(x["Precision"], x["Recall"]), axis = 1)


In [18]:
data

,id,summaries,exact_matched_patt_contextual,BERT_Tags,Eng_Label,TP_entries_eng,FP_entries_eng,FN_entries_eng,FP corrected,TP,FP,FN,Precision,Recall,f_score
0,-1zMVjZsVTSaqEUPGqaTqA_0000,i have a masters degree in computational finan...,[java],['java'],['java'],[java],[],[],[],1,0,0,1.0,1.0,1.000000
1,0BEpuKXd00LyMNQDgcqUZw_0000,i am a strategic hr leader with single and mul...,[courion],[],[],[],[],[],[],0,0,0,0.0,0.0,0.000000
2,0f9VnfZFwZiyd30xGBK9Ig_0000,multitalented creative marketing and design pr...,[adobe],"['adobe', 'beacon']",['adobe'],[adobe],[beacon],[],[],1,1,0,0.5,1.0,0.666667
3,0GTKKIK9kj-FVYLEkrqqcA_0000,it professional with an overall 16 years of te...,[wipro],[],[],[],[],[],[],0,0,0,0.0,0.0,0.000000
4,0iMM991ETr11zJf6ABimew_0000,an innovative resultdriven leader with a prove...,[paypal],[],[],[],[],[],[],0,0,0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,zun1u5pEP6XGEgB8NnH5Cw_0000,after teaching at hogeschool rotterdam i found...,[adobe],[],['adobe'],[],[],[adobe],[],0,0,1,0.0,0.0,0.000000
496,zYOI4JmmTDiT023cU9g7XA_0000,having spent 8 months in 2011 working with dan...,[rsm],[],[],[],[],[],[],0,0,0,0.0,0.0,0.000000
497,ZzIyes8qbSx2AXxe4cSPlA_0000,after 15 years of successful sales and recruit...,[casa],[],[],[],[],[],[],0,0,0,0.0,0.0,0.000000
498,NaN,experienced information technology professiona...,[teradata],[],[],[],[],[],[],0,0,0,0.0,0.0,0.000000


In [19]:
data.to_csv("BERT_ENG_500.csv")